<a href="https://colab.research.google.com/github/andreidm92/Agents_in_code/blob/main/practice/Lesson_09_sql_warehouse_full.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🎯 Day 9 — Warehouse Supervisor: SQL Data Analyst Bot

---

## 📘 Теория (≈45 мин)

### 📌 Часть 1 — LlamaIndex: `SQLDatabaseReader`

**Назначение**: загрузка таблиц из SQL-баз данных как `Document`-объектов для последующей индексации и RAG.

**Импорт**:
```python
from llama_index.readers.database import SQLDatabaseReader
```

**Создание объекта**:
```python
from sqlalchemy import create_engine
from llama_index.readers.database import SQLDatabaseReader

engine = create_engine("sqlite:///example.db")  # или postgres, mysql и т.п.
reader = SQLDatabaseReader(engine)
documents = reader.load_data()
```

**Параметры**:
- `engine`: SQLAlchemy Engine
- `database_name` (опционально)
- `include_table_schema` (bool): добавлять описание схем таблиц
- `sample_rows_in_table_info` (int): сколько строк показать для каждого table_info
- `show_table_names_only` (bool): только список таблиц

**Применяется для**: загрузки фактических данных с WMS/ERP систем, создания агентов‑аналитиков по логистике.

---

### 📌 Часть 2 — LangGraph: LangSmith Metrics

**Цель**: трассировка и анализ поведения агентов через LangSmith: от логов до A/B тестов и визуализации.

**Компоненты**:
- `langsmith.Client()` — клиент SDK
- Метки (tags), Трейсы (traces), ранкинги
- Интеграция с LangChain и LangGraph без дополнительного кода (авто‑hook)

**Пример включения логирования**:
```python
from langsmith import traceable

@traceable(name="sql_query_step")
def run_query(query):
    return execute_query(query)
```

**Используется для**:
- Оптимизации агентов, выявления слабых мест
- A/B тестов (две версии промпта или кода)
- Визуализация поведения агентов

---

### 📌 Часть 3 — LangGraph: Human-in-the-Loop Node

**Назначение**: вставка шага, где человек подтверждает или дополняет решение агента.

```python
def sql_confirmation_node(state):
    print("🔍 SQL-запрос от агента:")
    print(state["sql"])
    input("Подтвердите запрос и нажмите Enter...")
    return state
```

Или встроенный input:

```python
query = "Какие товары есть на складе меньше чем по 100 штук?"
input(f"🤖 Агент предложил запрос:\n'{query}'\n\n✅ Подтвердите и нажмите Enter...")
```

---

## 💡 Ключевые ссылки
- [SQLDatabaseReader – Docs](https://docs.llamaindex.ai/en/stable/api_reference/readers/integrations/sql.html)
- [LangSmith Observability](https://docs.smith.langchain.com)

---

## 🛠 Практика (≈75 мин)

### 📌 Мини-проект: Агент‑аналитик остатков на складе

**Описание**: Агент принимает вопрос и генерирует SQL-запрос к WMS-базе (в SQLite), анализирует остатки, возвращает результат с пояснением.

---

### ✅ Шаги:

1. 📦 Загрузить WMS SQLite DB (сгенерируем mock-данные)
2. 📊 Подключить SQLDatabaseReader → получить Documents
3. 🧠 Построить векторный индекс (VectorStoreIndex)
4. 🤖 Сделать агент-чатбот: получает вопрос → извлекает релевантные фрагменты → формирует SQL-запрос → интерпретирует результат
5. 📉 Добавить LangSmith trace логирование
6. 🔁 Включить Human-in-the-Loop подтверждение запроса
7. 🧪 Написать 2-3 теста (unit-test-like)

---

🔗 Используй ноутбук [`day09_sql_warehouse_HIL.ipynb`](sandbox:/mnt/data/day09_sql_warehouse_HIL.ipynb)


In [2]:
!pip install llama-index langchain langsmith sqlalchemy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 60.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.9/265.9 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 720.4/720.4 kB 43.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 303.4/303.4 kB 23.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 11.1 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.78.1
    Uninstalling openai-1.78.1:
      Successfully uninstalled openai-1.78.1


In [4]:
!pip install llama-index-readers-database

In [1]:
# 📌 Генерация mock-таблицы WMS (остатки)
import sqlite3

conn = sqlite3.connect("wms.db")
cursor = conn.cursor()

cursor.execute("DROP TABLE IF EXISTS inventory")
cursor.execute('''
CREATE TABLE inventory (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    product_name TEXT,
    quantity INTEGER,
    updated_at TEXT
)
''')

products = [("Шурупы", 1200, "2024-05-21"),
            ("Гайки", 800, "2024-05-22"),
            ("Болты", 300, "2024-05-23"),
            ("Петли", 50, "2024-05-20")]
cursor.executemany("INSERT INTO inventory (product_name, quantity, updated_at) VALUES (?, ?, ?)", products)
conn.commit()
conn.close()

In [5]:
from sqlalchemy import create_engine
from llama_index.readers.database import DatabaseReader

engine = create_engine("sqlite:///wms.db")
reader = DatabaseReader(engine=engine)

# Пример SQL-запроса
query = "SELECT product_name, quantity FROM inventory WHERE quantity < 100"
documents = reader.load_data(query=query)

for doc in documents:
    print(doc.text)



product_name: Петли, quantity: 50


In [3]:
import os, getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Вставь OpenAI API ключ: ")

Вставь OpenAI API ключ: ··········


In [11]:
# 📌 Построение индекса и запуск query engine
from llama_index.core import VectorStoreIndex
index = VectorStoreIndex.from_documents(documents)
engine = index.as_query_engine()

response = engine.query(
    "Какой товар есть в базе и в каком количестве, если он числится в списке с количеством менее 100?"
)


print(response)



Товар "Петли" есть в базе и его количество составляет 50.


In [13]:
response = engine.query("Что ты можешь сказать о товаре Петли?")

print(response)

The product "Петли" has a quantity of 50 available.


In [8]:
# TODO: Включи LangSmith наблюдаемость, если используешь langsmith.run или @traceable
from langsmith import traceable
@traceable(name="sql_answer")
def query_inventory(q): return engine.query(q)

📌 Что делает @traceable
@traceable — это декоратор из langsmith, который позволяет:

логировать вызовы функций

отслеживать шаги агента

анализировать поведение модели

передавать результаты в LangSmith Dashboard (если настроен)

✅ Как его использовать
Убедись, что у тебя есть LangSmith API-ключ

import os
os.environ["LANGCHAIN_API_KEY"] = "sk-..."
Импортируй и оберни функцию:


from langsmith import traceable

@traceable(name="sql_answer")
def query_inventory(q):
    return engine.query(q)
Вызов:

response = query_inventory("Какие товары остались меньше чем 100 штук?")
print(response)
Результат будет виден в LangSmith UI (https://smith.langchain.com)

🔒 Без API-ключа?
Если ты не используешь LangSmith сейчас, просто оставь как комментарий или убери. Это опционально.



In [9]:
print(response.response)


The product is "Петли" and the quantity is 50.


In [14]:
# 🔎 Проверка, что в ответе упоминается нужный товар
assert "Петли" in response.response, "❌ Товар 'Петли' не найден в ответе"

# 🔢 Проверка, что упоминается количество
assert any(x in response.response for x in ["50", "50 штук", "количество 50"]), "❌ Количество не найдено"

print("✅ Все тесты прошли.")


✅ Все тесты прошли.


### 📌 Часть 3 — LangGraph: Human-in-the-Loop Node


Human-in-the-Loop (HITL) из LangGraph, чтобы вставить ручную проверку на этапе исполнения агента. Это особенно полезно, когда:

Агент генерирует SQL-запрос

Перед выполнением нужно подтвердить или отредактировать его вручную

✅ Пример полного применения в LangGraph

In [16]:
!pip install langgraph


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.9/154.9 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.0/50.0 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 216.5/216.5 kB 14.5 MB/s eta 0:00:00


In [19]:
from langgraph.graph import StateGraph
from typing import TypedDict

# 👇 Описание структуры состояния
class SQLState(TypedDict):
    sql: str
    result: str

# 👤 Узел HITL
def confirm_sql_node(state: SQLState) -> SQLState:
    print("🔍 SQL-запрос от агента:")
    print(state["sql"])
    input("✅ Нажми Enter для подтверждения")
    return state

# 👷 Примерный pipeline
graph = StateGraph(SQLState)
graph.add_node("generate_sql", lambda state: {"sql": "SELECT * FROM inventory WHERE quantity < 100"})
graph.add_node("confirm_sql", sql_confirmation_node)
graph.add_node("run_sql", lambda state: {"result": "Петли, 50"})

graph.set_entry_point("generate_sql")
graph.add_edge("generate_sql", "confirm_sql")
graph.add_edge("confirm_sql", "run_sql")
graph.set_finish_point("run_sql")

# 🧪 Запуск
app = graph.compile()
output = app.invoke({})
print(output)


🔍 SQL-запрос от агента:
SELECT * FROM inventory WHERE quantity < 100
✅ Подтвердите и нажмите Enter...
{'sql': 'SELECT * FROM inventory WHERE quantity < 100', 'result': 'Петли, 50'}
